<a name="top"></a>
<div style="width:1000 px">

<div style="float:right; width:98 px; height:98px;">
<img src="https://raw.githubusercontent.com/Unidata/MetPy/master/src/metpy/plots/_static/unidata_150x150.png" alt="Unidata Logo" style="height: 98px;">
</div>

<h1>Downloading Model Output Plotting</h1>
<h3>Unidata Python Workshop</h3>

<div style="clear:both"></div>
</div>

<hr style="height:2px;">

<div style="float:right; width:250 px"><img src="https://unidata.github.io/siphon/latest/_static/siphon_150x150.png" alt="TDS" style="height: 200px;"></div>

### Questions
1. What is the netCDF Subset Service (NCSS)?
1. How can I use Siphon to make an NCSS request?
1. How do I plot gridded fields using CartoPy?

### Objectives
1. <a href="#ncss">Use siphon to make a request using NCSS</a>
1. <a href="#xarray">Creating an XArray Data Array</a>
1. <a href="#plotting">Make a Map</a>
1. <a href="#pointrequest">Requesting for a single point</a>

<a name="ncss"></a>
## 1. What is NCSS?

In [ ]:
# Resolve the latest GFS dataset
import metpy
from siphon.catalog import TDSCatalog

# Set up access via NCSS
gfs_catalog = ('http://thredds.ucar.edu/thredds/catalog/grib/NCEP/GFS/'
               'Global_0p5deg/catalog.xml?dataset=grib/NCEP/GFS/Global_0p5deg/Best')
cat = TDSCatalog(gfs_catalog)
ncss = cat.datasets[0].subset()

We can see what variables are available from ncss as well:

In [ ]:
ncss.variables

From here, we can build a query to ask for the data we want from the server.

In [ ]:
from datetime import datetime, timedelta

# Create a new NCSS query
query = ncss.query()

# Request data in netCDF format
query.accept('netcdf')

# Ask for our variable
query.variables('Temperature_isobaric')

# Ask for the 500 hPa surface
query.vertical_level(50000)

# Set the time range of data we want
now = datetime.utcnow()
query.time_range(now, now + timedelta(days=1))

# Set the spatial limits
query.lonlat_box(west=-110, east=-45, north=50, south=10)

# get the data!
data = ncss.get_data(query)

In [ ]:
data

<a href="#top">Top</a>
<hr style="height:2px;">

<a name="xarray"></a>
## 2. Creating an XArray Data Array

We know that the declarative plotting system works really well with XArray data arrays, so we need to get our NetCDF data into a data array. XArray makes this relatively easy with the NetCDF4DataStore backend.

In [ ]:
from xarray.backends import NetCDF4DataStore
import xarray as xr

# We need the datastore so that we can open the existing netcdf dataset we downloaded
ds = xr.open_dataset(NetCDF4DataStore(data))

In [ ]:
ds

We can explore the data with XArray, but for now we're most interested in which time steps are available so we can pick one to plot. Feel free to explore some of the other coordinates, data, and attributes of the data though!

In [ ]:
ds.time

We could do some rather ugly manipulation of numpy datetimes here, but instead we just say when we want the plot to be valid and MetPy will do it's best by plotting the nearest available time to our request!

In [ ]:
# Create the desired plot time
plot_time = now + timedelta(hours=12)

<a href="#top">Top</a>
<hr style="height:2px;">

<a name="plotting"></a>
## Make a Map

In [ ]:
from metpy.plots.declarative import *
from metpy.units import units

Let's start out by making a very basic plot of the grid - we just want to get an idea of what's here.

In [ ]:
# Make a basic image plot
img = ImagePlot()
img.data = ds
img.field = 'Temperature_isobaric'
img.level = 500 * units.hPa
img.time = plot_time

In [ ]:
# Create the map panel and add the plot to it
panel = MapPanel()
panel.plots = [img]

In [ ]:
# Create a panel container and add the panel to it
pc = PanelContainer()
pc.panels = [panel]
pc.show()

Now that we have a basic example working, we can get a little fancier and start dressing up our plot. Remember building up is easier to troubleshoot than writing it all in one shot! We do have to recreate the `ImagePlot`, `MapPanel`, and `PanelContainer` each time though due to how things are working under the hood to make declarative plotting possible.

In [ ]:
# Make a basic image plot
img = ImagePlot()
img.data = ds
img.field = 'Temperature_isobaric'
img.level = 500 * units.hPa
img.time = plot_time
img.colormap = 'coolwarm'
img.colorbar = 'horizontal'

# Create the map panel and add the plot to it
panel = MapPanel()
panel.plots = [img]
panel.layers = ['coastline', 'borders', 'states']

# Create a panel container and add the panel to it
pc = PanelContainer()
pc.panels = [panel]
pc.show()

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
        <li>Extend the plot above by plotting contours of 500 hPa geopotential heights onto the temperature image plot. (You'll need to get the data.)</li>
        <li>Add a title to the plot with the correct time.</li>
         <li>Increase the figure size for a better look.</li>
    </ul>
</div>

In [ ]:
# Set up an NCSS query from thredds using siphon
# YOUR CODE GOES HERE

# Download data using NCSS and create Data Array
# YOUR CODE GOES HERE

# Get the first time step as a datetime
# YOUR CODE GOES HERE

# Make a temperature image plot
# YOUR CODE GOES HERE

# Make geopotential contour plot
# YOUR CODE GOES HERE

# Create a panel container and add the panel to it
# YOUR CODE GOES HERE

<div class="alert alert-info">
    <b>SOLUTION</b>
</div>

In [ ]:
# %load solutions/dec_contour_map.py

We can even add wind barbs to the plot! We have to remove the level from our query though as winds are on a different coordinate that would be in hPa. That's okay though, when we plot we can select the correct level!

In [ ]:
# Set up an NCSS query from thredds using siphon
query = ncss.query()
query.accept('netcdf4')
query.variables('Temperature_isobaric', 'Geopotential_height_isobaric',
                'u-component_of_wind_isobaric', 'v-component_of_wind_isobaric')
now = datetime.utcnow()
query.time_range(now, now + timedelta(days=1))
query.lonlat_box(west=-110, east=-45, north=50, south=10)

# Download data using NCSS and create Data Array
data = ncss.get_data(query)
ds = xr.open_dataset(NetCDF4DataStore(data))

# Create the desired plot time
plot_time = now + timedelta(hours=12)

In [ ]:
# Make a temperature image plot
tmp_img = ImagePlot()
tmp_img.data = ds
tmp_img.field = 'Temperature_isobaric'
tmp_img.level = 500 * units.hPa
tmp_img.time = plot_time
tmp_img.colormap = 'coolwarm'
tmp_img.colorbar = 'horizontal'

# Make geopotential contour plot
geopot_cnt = ContourPlot()
geopot_cnt.data = ds
geopot_cnt.field = 'Geopotential_height_isobaric'
geopot_cnt.level = 500 * units.hPa
geopot_cnt.time = plot_time

# Add wind barbs
barbs = BarbPlot()
barbs.data = ds
barbs.level = 500 * units.hPa
barbs.time = plot_time
barbs.field = ('u-component_of_wind_isobaric', 'v-component_of_wind_isobaric')

# Create the map panel and add the plot to it
panel = MapPanel()
panel.plots = [tmp_img, geopot_cnt, barbs]
panel.layers = ['coastline', 'borders', 'states']
panel.title = plot_time.strftime('%Y-%m-%d at %H:%MZ')

# Create a panel container and add the panel to it
pc = PanelContainer()
pc.panels = [panel]
pc.size = (10, 8)
pc.show()

There! We've got our plot and are ready to publish right? Wrong! There are obviously too many wind barbs there. We need to skip some (naive downsample).

In [ ]:
# Make a temperature image plot
tmp_img = ImagePlot()
tmp_img.data = ds
tmp_img.field = 'Temperature_isobaric'
tmp_img.level = 500 * units.hPa
tmp_img.time = plot_time
tmp_img.colormap = 'coolwarm'
tmp_img.colorbar = 'horizontal'

# Make geopotential contour plot
geopot_cnt = ContourPlot()
geopot_cnt.data = ds
geopot_cnt.field = 'Geopotential_height_isobaric'
geopot_cnt.level = 500 * units.hPa
geopot_cnt.time = plot_time

# Add wind barbs
barbs = BarbPlot()
barbs.data = ds
barbs.level = 500 * units.hPa
barbs.time = plot_time
barbs.field = ('u-component_of_wind_isobaric', 'v-component_of_wind_isobaric')
barbs.skip = (5, 5)

# Create the map panel and add the plot to it
panel = MapPanel()
panel.plots = [tmp_img, geopot_cnt, barbs]
panel.layers = ['coastline', 'borders', 'states']
panel.title = plot_time.strftime('%Y-%m-%d at %H:%MZ')

# Create a panel container and add the panel to it
pc = PanelContainer()
pc.panels = [panel]
pc.size = (10, 8)
pc.show()


<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
        <li>Set the area of the MapPanel to match our request area.</li>
    </ul>
</div>

In [ ]:
# COPY THE MAP FROM ABOVE AND MODIFY IT HERE

<div class="alert alert-info">
    <b>SOLUTION</b>
</div>

In [ ]:
# %load solutions/dec_map_area.py

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
        <li>Create a new request/plot for different data from the model and create a new plot from scratch. Try to not copy/paste, but create from scratch or by typing copy at a minimum to help get your code muscles trained.</li>
    </ul>
</div>

In [ ]:
# CREATE A DATA REQUEST

In [ ]:
# CREATE A MAP

<div class="alert alert-info">
    <b>SOLUTION</b>
</div>

In [ ]:
# %load solutions/dec_any_map.py

<a href="#top">Top</a>
<hr style="height:2px;">